In [2]:
!pip install pandas


In [3]:
!pip install bs4

In [4]:
!pip install requests 

In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [63]:
import threading


In [83]:
data = []

def find_datas(i):
    #gets all our data within the table and adds it to our dataframe
#     for i in range(1,6):
        url = f'https://maktabkhooneh.org/learn/%D8%B2%D8%A8%D8%A7%D9%86%D9%87%D8%A7%DB%8C-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D9%86%D9%88%DB%8C%D8%B3%DB%8C-mk76/?p={i}&'
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')

        courses = soup.find_all('div', class_ = 'side-margin bottom-margin js-filter-item active-item')

        for course in courses:
            #Title:
            row_data = []
            try:
                row_data.append(course['title'])
            except:
                row_data.append('')
            #Teacher:
            try:
                row_data.append(course.find('div', class_ = 'course-card__teacher').text.strip())
            except:
                row_data.append('')
            #School:
            try:
                row_data.append(course.find('div', class_ = 'course-card__uni-title').text.strip())
            except:
                row_data.append('')
            #Link:
            try:
                row_data.append('https://maktabkhooneh.org'+course.find('a', class_ = 'course-card__wrapper')['href'])
            except:
                row_data.append('')
            #Price
            price_url ='https://maktabkhooneh.org'+course.find('a', class_ = 'course-card__wrapper')['href']+'purchase/?type=FULL'
            price_page = requests.get(price_url)
            price_soup = BeautifulSoup(price_page.text, 'html.parser')
            try:
                p = price_soup.find('span', {'id' : 'final_value'}).text
                row_data.append(int(p.split()[0].replace(',','')))
            except:
                row_data.append(0)
            #Time:
            time_url ='https://maktabkhooneh.org' + course.find('a', class_ = 'course-card__wrapper')['href']
            time_page = requests.get(time_url)
            time_soup = BeautifulSoup(time_page.text , 'html.parser')
    #         print(time_url,'8'*10 )
            try:
    #             print(time_url , '\n')
                t_list=[]
                t_list = time_soup.find_all('div', class_='course-information__value')#.text.strip()# age .text nabashe nemire too exception
    #             print(t_list)
                row_data.append(t_list[0].text)
            except:
                row_data.append(0)
    #             print('exception!')
            #Sessions:
            try:
                sc_list = time_soup.find_all('div' , class_='h-slider__wrapper')
                sc= len(sc_list)
                row_data.append(sc)
            except:
                row_data.append(0)
    #             print('exception!')

            data.append(row_data)


In [84]:
threads=[]
for i in range(5):
     t = threading.Thread(target=find_datas , args=[i])
     t.start()
     threads.append(t)

for thread in threads:
    thread.join()
    


In [85]:
headers = ['عنوان', 'مدرس', 'آموزشگاه', 'لینک', 'قیمت', 'ساعت', 'جلسات']
#Creates a dataframe using the column headers from our table
df = pd.DataFrame(data, columns = headers)
fdf=df.sort_values(by =['عنوان', 'مدرس','آموزشگاه','لینک','قیمت','جلسات','ساعت'])

In [94]:
fdf.to_csv('maktabkhooneh_99522329.csv')

In [91]:
len(fdf)
#75 = 15 * 5
# baeed midoonam ke dochaare race condition nashodim!

75

In [99]:
fdf.to_excel('maktabkhooneh_99522329.xlsx')

In [95]:
!pip install openpyxl
